In [1]:
import sqlite3

In [2]:
from datetime import datetime


In [3]:
conn = sqlite3.connect("contact_management.db")

In [4]:
cur = conn.cursor()

In [7]:
cur.execute("""CREATE TABLE IF NOT EXISTS contact (
               fname TEXT,
               lname TEXT,
               contact TEXT,
               email TEXT,
               city TEXT,
               CHECK (email LIKE '%_@_%._%')
            )""")


In [8]:
cur.execute("""CREATE TABLE IF NOT EXISTS log (
               operation TEXT,
               fname TEXT,
               lname TEXT,
               old_contact TEXT,
               new_contact TEXT,
               timestamp DATETIME
            )""")

In [9]:
def insert_contact(fname, lname, contact, email, city):
    cur.execute("INSERT INTO contact VALUES (?, ?, ?, ?, ?)", (fname, lname, contact, email, city))
    conn.commit()
    log_operation("insert", fname, lname, old_contact=None, new_contact=contact)

In [10]:
def update_contact(fname, lname, new_contact):
    cur.execute("SELECT contact FROM contact WHERE fname = ? AND lname = ?", (fname, lname))
    old_contact = cur.fetchone()[0]
    cur.execute("UPDATE contact SET contact = ? WHERE fname = ? AND lname = ?", (new_contact, fname, lname))
    conn.commit()
    log_operation("update", fname, lname, old_contact, new_contact)

In [11]:
def delete_contact(fname, lname):
    cur.execute("SELECT contact FROM contact WHERE fname = ? AND lname = ?", (fname, lname))
    old_contact = cur.fetchone()[0]
    cur.execute("DELETE FROM contact WHERE fname = ? AND lname = ?", (fname, lname))
    conn.commit()
    log_operation("delete", fname, lname, old_contact, new_contact=None)

In [12]:
def search_contact(fname, lname):
    cur.execute("SELECT * FROM contact WHERE fname = ? AND lname = ?", (fname, lname))
    return cur.fetchall()

In [13]:
def log_operation(operation, fname, lname, old_contact, new_contact):
    timestamp = datetime.now()
    cur.execute("INSERT INTO log VALUES (?, ?, ?, ?, ?, ?)",
                (operation, fname, lname, old_contact, new_contact, timestamp))
    conn.commit()

In [14]:
insert_contact("John", "Doe", "1234567890", "john@example.com", "New York")
update_contact("John", "Doe", "9876543210")
delete_contact("John", "Doe")
print(search_contact("John", "Doe"))


[]


In [15]:
conn.close()